In [ ]:
from lxml import html
import lxml
import time
import requests
import pandas as pd
import sqlite3
import re

In [ ]:
date_jour = '2017-12-31'
OUTPUT_PATH = '../_outputs/'
database_file = OUTPUT_PATH + 'chefs_{}.db'.format(date_jour)
conn = sqlite3.connect(database_file)

In [ ]:
df_source = pd.read_sql_query("select * from index_chefs;", conn)
df_source.head()

In [ ]:
def getChefPage(chef_id):
    url = 'https://labelleassiette.fr/' + chef_id
    response = requests.get(url)
    content_source = response.text
    return [chef_id, html.document_fromstring(content_source)]

In [ ]:
docs = []
for index, row in df_source.iterrows():
    docs.append(getChefPage(row['chef']))

In [ ]:
len(docs)

In [ ]:
docs[0][1]

In [ ]:
# https://stackoverflow.com/questions/6937525/escaping-xpath-literal-with-python
def toXPathStringLiteral(s):
    if "'" not in s: return "'%s'" % s
    if '"' not in s: return '"%s"' % s
    return "concat('%s')" % s.replace("'", "',\"'\",'")

In [ ]:
def get_rows(doc):
    
    menus = doc.xpath('//h3[@class="menu-title"]/text()')
    
    descriptions = doc.xpath('//p[@class="menu-resume extract"]/text()')
    
    tags = []
    descriptions = []
    
    for m in menus:
        if(m):
            tags.append(
                doc.xpath(
                    '//h3[text()=%s]/following-sibling::ul[@class="tags"]/li/text()' 
                    % toXPathStringLiteral(m)
                )
            )
            
            descriptions.append(
                doc.xpath(
                    '//h3[text()=%s]/../following-sibling::div[@class="menu-details full"]//p[@class="dish"]/text()' 
                    % toXPathStringLiteral(m)
                )
            )
        
        else:
            tags.append('NA')
    
    scripts = doc.findall('.//script')
    prices = []
    item = 0
    for s in scripts:
        if 'chef.menus[{}]'.format(item) in s.text_content():
            prices.append(s.text_content())
            item += 1  
    # print(prices[1])
    return menus, descriptions, tags, prices

In [ ]:
columns = ['chef', 'menu', 'description', 'tags', 'prices']
df_output = pd.DataFrame(columns=columns)

In [ ]:
idx = 0
for doc in docs:
    menus, descriptions, tags, prices = get_rows(doc[1])
    chef = doc[0]
    # print(chef)
    idxm = 0
    for m in menus:
        # print(m)
        df_output.loc[idx] = [chef, m, descriptions[idxm], tags[idxm], prices[idxm]]
        idx +=1
        idxm +=1

In [ ]:
df_output.head(50)

In [ ]:
df_output.to_excel(OUTPUT_PATH + 'chefs_{}_menus.xlsx'.format(date_jour), index=False, header=True)